<a href="https://colab.research.google.com/github/MicaelaRomeroC3/sotto/blob/main/Sottotempo_Limp_Registro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Evaluaciones prueba Decathlon.xlsx'

# Cargar el archivo Excel
df = pd.read_excel(file_path)

# Mostrar las primeras filas del archivo
df.head()


,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C13. CALIDAD DE LA RESPUESTA.COND.LA RESPUESTA PROPORCIONADA DURANTE LA CONVERSACIÓN DE WHATSAPP ES FALSA. MUESTRA NULO CONOCIMIENTO (ENTENDIMIENTO) DE LA CONSULTA O PRODUCTO.,C13. CALIDAD DE LA RESPUESTA.COND.EL CHAT DE WHATSAPP NO HA SIDO ATENDIDO TRAS PASARSE EL PLAZO DE 5 MINUTOS Y NO RECIBIR RESPUESTA POR PARTE DE UN ASESOR.,C14. FUNCIONALIDAD.COND.EL LINK FACILITADO POR EL ASESOR LLEVA A UNA PÁGINA CON LA SOLUCIÓN A LA CONSULTA PLANTEADA POR EL CLIENTE.,C14. FUNCIONALIDAD.COND.EL LINK FACILITADO POR EL ASESOR LLEVA A UNA PÁGINA DIFERENTE A LO CONSULTA DESCRITA POR EL CLIENTE.,C15. GESTIÓN CORRECTA DE LA SOLICITUD,C15. GESTIÓN CORRECTA DE LA SOLICITUD.COND.EL ASESOR SÍ DERIVA AL CANAL CORRECTO DONDE EL CLIENTE PUEDE SOLVENTAR LA CONSULTA.,C15. GESTIÓN CORRECTA DE LA SOLICITUD.COND.EL ASESOR NO DERIVA AL CANAL CORRECTO DONDE EL CLIENTE PUEDE SOLVENTAR LA CONSULTA.,C16. SOLUCIÓN AL PRIMER CONTACTO (FCR),C16. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y NO ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.,C16. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.NO SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.
0,101S_969-OP_11_291_20250226H_12_03,LLAMADA PRUEBA 1,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,
1,101S_969-OP_11_291_20250226H_12_07,LLAMADA PRUEBA 2,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,
2,101S_969-OP_11_291_20250226H_12_11,LLAMADA PRUEBA 3,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,
3,101S_969-OP_11_291_20250226H_12_15,LLAMADA PRUEBA 4,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,
4,101S_969-OP_11_291_20250226H_12_18,LLAMADA PRUEBA 5,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,


# **Extraer el canal de la columna SERVICIO**

In [5]:
# Lista de palabras clave
servicios_validos = ['Teléfono', 'Mail/Formulario', 'Web', 'Twitter', 'Facebook', 'Chat Web', 'Whatsapp']

# Función para extraer la palabra clave válida
def limpiar_servicio(servicio):
    for palabra in servicios_validos:
        if palabra in servicio:
            return palabra
    return None  # Retorna None si no encuentra ninguna palabra válida

# Aplicar la función a la columna "Servicio"
df['Servicio Limpio'] = df['SERVICIO'].apply(limpiar_servicio)

# Mostrar el DataFrame con la nueva columna
print(df['Servicio Limpio'])

0            Teléfono
1            Teléfono
2            Teléfono
3            Teléfono
4            Teléfono
5            Teléfono
6     Mail/Formulario
7     Mail/Formulario
8     Mail/Formulario
9     Mail/Formulario
10    Mail/Formulario
11    Mail/Formulario
12                Web
13                Web
14                Web
15                Web
16                Web
17                Web
18            Twitter
19            Twitter
20            Twitter
21           Facebook
22           Facebook
23           Facebook
24                Web
25                Web
26                Web
27           Whatsapp
28           Whatsapp
29           Whatsapp
Name: Servicio Limpio, dtype: object


**Solo si se requiere descargar ejecutar la siguiente casilla**

In [ ]:
# Guardar el DataFrame modificado como archivo Excel
df.to_excel('Evaluaciones prueba Decathlon limpio.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Evaluaciones prueba Decathlon limpio.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Crear hoja REGISTRO**

In [9]:
# Renombramos columnas para facilitar el trabajo
df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)

# Mapear los canales a tipo contacto
def map_tipo_contacto(canal):
    canal = canal.lower()
    if canal in ['facebook', 'twitter']:
        return 'Redes Sociales'
    elif canal in ['chat web', 'whatsapp']:
        return 'Chat'
    else:
        return canal

# Aplicamos el mapeo
df['tipo_contacto'] = df['servicio_limpio'].apply(map_tipo_contacto)

# Capitalizamos la primera letra de cada tipo_contacto
df['tipo_contacto'] = df['tipo_contacto'].str.capitalize()

# Formateamos la fecha
df['fecha'] = pd.to_datetime(df['fecha_evaluación']).dt.date

# Columna fija
df['tipo_de_acción'] = 'evaluada'

# Agrupamos por fecha, empresa y tipo_contacto (canal)
result = df.groupby(['fecha', 'empresa', 'tipo_contacto']).size().reset_index(name='cantidad')

# Añadimos columna de tipo_de_acción (ya está aplicada arriba, pero por si acaso)
result['tipo_de_acción'] = 'Evaluada'

# Reordenamos columnas
result = result[['fecha', 'empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostramos resultado
result.head()

<ipython-input-9-d42fe317b818>:21: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['fecha'] = pd.to_datetime(df['fecha_evaluación']).dt.date


,fecha,empresa,tipo_contacto,tipo_de_acción,cantidad
0,2025-02-26,Decathlon,Chat,Evaluada,3
1,2025-02-26,Decathlon,Mail/formulario,Evaluada,6
2,2025-02-26,Decathlon,Redes sociales,Evaluada,6
3,2025-02-26,Decathlon,Teléfono,Evaluada,6
4,2025-02-26,Decathlon,Web,Evaluada,9
